# Pelican Nebua Data Extraction and 3D Velocity calculations
*Eric G. Suchanek, Ph.D. 9/29/2019*

This code performs a Gaia rectangular search from within the Pelican Nebula and extracts
* non-null fields pmra, pmdec, with parallax > 0. The query strings are stored in egs.py, and can be easily modified.

3d velocities were calculated per the formulae discussed in: http://www.astronexus.com/a-a/motions-long-term. These values were then written out to a csv file containing the query results. The approximate center of NGC 7000 is used for the query center. The approximate distance to the nebula is 675pc.

The code is sufficiently general to be re-used to perform stellar extractions against the Gaia DR2 database, perform pm angle and 3d velocity calculations, and write the output to a well-defined file in .csv format. Simply set the center point and extents as well as the project_name variables. 


In [2]:
# Author: Eric G. Suchanek, Ph.D., MIRA#
# Setup the libraries and initialize any global variables.
# This code must be run first prior to any other cells.
#
# Suppress warnings. Comment this out if you wish to see the warning messages
import warnings
warnings.filterwarnings('ignore')

# We go offline with cufflinks first, since this is all local
#
import plotly.offline

from time import time
from pandas import DataFrame

from mirapy import *
import mirapy.egs as egs

#from mirapy import *
from mirapy.utils import pprint_elapsed

# global variable used for pprinting the elapsed time. one of linux, imac, pc
_sys = imac

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [7]:
# Program to extract stars based on systematic variation of extraction size, 
# centered on NGC 7000 (the North American Nebula)

# specify the absolute path prefix (relative to the User's HOME directory), for 
# the location of the extracted stars will need to use this when reading as well!

"""import sys

# you have to use escape on Windows due to the assinine path formats
if sys.platform == 'win32':
    project_dir = "\\MIRA\\data\\Pel_subset\\"
else:
    project_dir = "/MIRA/data/Pel_subset/"
"""
project_dir = "/MIRA/data/Pel_subset/"

file_prefix = "PelPM_"

# Center of Pelican Nebula
ra_center = "20h50m49s" 
dec_center = "44d20m59s" 

star_count_list = []
size_list = []
time_list = []

start = time()

# loop over various extraction sizes. note we are doing square extractions so width = height
for size in ["10m", "15m", "20m", "30m", "45m", "60m"]:
    print("Starting batch run for query size:", size, "x", size)
    star_count, secs, output_filename = egs.extract_Gaia_stars(ra_center, dec_center,
                                                               size, size,
                                                               project_dir, file_prefix,
                                                               Filtered=True,verbose=False)
    print('-- Returned:', star_count, 'stars')
    pprint_elapsed(start,arch=_sys)
    print(' ')
    star_count_list.append(star_count)
    size_list.append(size)
    time_list.append(secs)

Starting batch run for query size: 10m x 10m
-- Returned: 468 stars
Elapsed: 0:00:01.907787 on iMac, 3.3 GHz i5, 32 GB DDR3 1867 MHz, OSX 10.15 beta
 
Starting batch run for query size: 15m x 15m
-- Returned: 975 stars
Elapsed: 0:00:05.121957 on iMac, 3.3 GHz i5, 32 GB DDR3 1867 MHz, OSX 10.15 beta
 
Starting batch run for query size: 20m x 20m
-- Returned: 1865 stars
Elapsed: 0:00:08.270829 on iMac, 3.3 GHz i5, 32 GB DDR3 1867 MHz, OSX 10.15 beta
 
Starting batch run for query size: 30m x 30m
-- Returned: 4390 stars
Elapsed: 0:00:16.304470 on iMac, 3.3 GHz i5, 32 GB DDR3 1867 MHz, OSX 10.15 beta
 
Starting batch run for query size: 45m x 45m
-- Returned: 10468 stars
Elapsed: 0:00:38.243240 on iMac, 3.3 GHz i5, 32 GB DDR3 1867 MHz, OSX 10.15 beta
 
Starting batch run for query size: 60m x 60m
-- Returned: 20185 stars
Elapsed: 0:01:21.827994 on iMac, 3.3 GHz i5, 32 GB DDR3 1867 MHz, OSX 10.15 beta
 


In [8]:
import plotly_express as px       
df = DataFrame()
df['starcount'] = star_count_list
df['Size'] = size_list
df['Seconds'] = time_list

title_str = "Star Count vs Extent"
fig = px.scatter(df, x="Size", y="starcount", template='plotly_dark', title=title_str)
fig.show()


title_str = "Query Time vs Extent"
fig2 = px.scatter(df, x="Size", y="Seconds", template='plotly_dark', title=title_str)
fig2.show()

In [5]:
# just trying out the Gaia class extract routine...
from astroquery.gaia import Gaia
import astropy.units as u

start = time()
pos = ra_center + " " + dec_center
result = Gaia.query_object_async(pos, width=.5*u.deg, height=.5*u.deg,verbose=True)
print('Stars returned:', len(result))
pprint_elapsed(start, _sys)

Launched query: 'SELECT DISTANCE(POINT('ICRS',ra,dec),                 POINT('ICRS',312.7041748767604,44.349727998765964)) AS dist, *                 FROM gaiadr2.gaia_source WHERE CONTAINS(                POINT('ICRS',ra,dec),                BOX('ICRS',312.7041748767604,44.349727998765964, 0.5, 0.5))=1                 ORDER BY dist ASC'
303 303
[('Date', 'Sat, 05 Oct 2019 03:17:57 GMT'), ('Server', 'Apache/2.4.6 (Red Hat Enterprise Linux) OpenSSL/1.0.2k-fips PHP/5.4.16 mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Set-Cookie', 'JSESSIONID=F5B14F9313E55E19B0B2A59627BD2F19; Path=/tap-server; HttpOnly'), ('Location', 'http://gea.esac.esa.int/tap-server/tap/async/1570245477477O'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'text/plain;charset=ISO-8859-1')]
job 1570245477477O, at: http://

In [ ]:
result


In [20]:
from astropy.coordinates import ICRS
def get_radial_vals2(star_list):
    """create a list containing non-zero Radial Velocities.

    The input star_list is a list object returned by job.results()

    Parameters
    ----------
    star_list : list returned by job.results()

    Returns
    -------
    v_gsr : list of coordinates containing non-zero radial_velicities
    converted to GSR
    """
    x = []
    x = [egs.rv_to_gsr(ICRS(ra=star['ra'] * u.deg, 
                        dec=star['dec'] * u.deg, 
                        radial_velocity=star['radial_velocity'] * u.km/u.s)).value 
         for star in star_list]
    return x


In [21]:
stars = egs.read_Gaia_stars(output_filename)
radial = get_radial_vals2(stars)

TypeError: string indices must be integers